Plots of wind vectors around Cape Ghir.

In [1]:
import os
import glob
import datetime
import netCDF4
import numpy as np
import filament
import logging
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
from importlib import reload
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [2]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.info("Starting")

## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [3]:
year, month, day = 2009, 8, 15
domain = (-19., -6., 25., 35.)
figdir = "../figures/wind/"
visibledir = "/data/Wind/visible/"
os.path.isdir(visibledir)

True

## Data

In [4]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

In [5]:
filelist = filament.get_filelist_url_quikscat(year, dayofyear)

In [6]:
filelistclean = []
for ifile, dataurl in enumerate(filelist):

    logger.info("Working on {}".format(dataurl))
    wind = filament.Wind()
    res = wind.read_from_quikscat(dataurl, domain)
    
    if res:
        filelistclean.append(dataurl)
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("CapeGhir-", dataurl.split("/")[-1].split(".")[0]))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(111, projection=myproj)
        wind.add_to_plot(fig, ax, domain=domain, cmap=plt.cm.hot_r,
                 clim=[0, 15], cbarloc=[0.6, 0.2, 0.25, 0.015])
        filament.decorate_map(ax, domain, np.arange(-19., -5.9, 3.), 
                              np.arange(25., 35.1, 2.))
        ax.add_feature(coast, linewidth=.2, color=".7")

        plt.savefig(figname, dpi=300, bbox_inches="tight")
        plt.close()

INFO:__main__:Working on https://opendap.jpl.nasa.gov/opendap/OceanWinds/quikscat/L2B12/v4.0/2009/227/qs_l2b_52886_v4.0_200908150119.nc
INFO:__main__:Working on https://opendap.jpl.nasa.gov/opendap/OceanWinds/quikscat/L2B12/v4.0/2009/227/qs_l2b_52887_v4.0_200908150300.nc
INFO:__main__:Working on https://opendap.jpl.nasa.gov/opendap/OceanWinds/quikscat/L2B12/v4.0/2009/227/qs_l2b_52888_v4.0_200908150441.nc
INFO:__main__:Working on https://opendap.jpl.nasa.gov/opendap/OceanWinds/quikscat/L2B12/v4.0/2009/227/qs_l2b_52889_v4.0_200908150622.nc


KeyboardInterrupt: 

### Plot only the saved files

In [7]:
datafilelist = sorted(glob.glob("/data/Wind/QuikSCAT/*.nc"))
datafilelist
smalldomain = (-15., -9., 27., 33.)

In [8]:
for datafile in datafilelist:
    wind = filament.Wind()
    wind.read_from_quikscat(datafile, smalldomain)
    
    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111, projection=myproj)
    wind.add_to_plot(fig, ax, domain=smalldomain, cmap=plt.cm.magma_r, 
                     clim=[5., 10.], date=wind.time.strftime("%Y-%m-%d"),
                     cbarloc=[0.6, 0.2, 0.25, 0.015])
    filament.decorate_map(ax, smalldomain, np.arange(-15., -.9, 1.), 
                          np.arange(27., 33.1, 1.))
    ax.add_feature(coast, linewidth=.2, color=".7")
    
    fname = "".join(("CapeGhir-", datafile.split("/")[-1].split(".")[0]))
    logger.info("Saving figure as {}".format(fname))
    figname = os.path.join(figdir, fname)
    plt.savefig(figname, dpi=300, bbox_inches="tight")
    # plt.show()
    plt.close()

INFO:__main__:Saving figure as CapeGhir-qs_l2b_52896_v4
INFO:__main__:Saving figure as CapeGhir-qs_l2b_53053_v4
INFO:__main__:Saving figure as CapeGhir-qs_l2b_53060_v4
INFO:__main__:Saving figure as CapeGhir-qs_l2b_53067_v4
